<a href="https://colab.research.google.com/github/BTrifonov/BTrifonov/blob/main/supply_network_design/supply_network_design_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supply Network Design 2

## Objective and Prerequisites

Take your supply chain network design skills to the next level in this example. We’ll show you how – given a set of factories, depots, and customers – you can use mathematical optimization to determine which depots to open or close in order to minimize overall costs.

This model is example 20 from the fifth edition of Model Building in Mathematical Programming, by H. Paul Williams on pages 275-276 and 332-333.

This example is of beginning difficulty; we assume that you know Python and have some knowledge of the Gurobi Python API and building mathematical optimization models.

**Download the Repository** <br />
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip).

---
## Problem Description

In this problem, we have six end customers, each with a known demand for a product.  Customer demand can be satisfied from a set of six depots, or directly from a set of two factories.  Each depot can support a maximum volume of product moving through it, and each factory can produce a maximum amount of product.  There are known costs associated with transporting the product, from a factory to a depot, from a depot to a customer, or from a factory directly to a customer. This extension provides the opportunity to choose which four of the six possible depots to open.  It also provides an option of expanding capacity at one specific depot.

Our supply network has two factories, in Liverpool and Brighton, that produce a product.  Each has a maximum production capacity:

| Factory | Supply (tons) |
| --- | --- |
| Liverpool | 150,000 |
| Brighton |  200,000 |

The product can be shipped from a factory to a set of six depots.  Each depot has a maximum throughput.  Depots don't produce or consume the product; they simply pass the product through to customers.

| Depot | Throughput (tons) |
| --- | --- |
| Newcastle | 70,000 |
| Birmingham | 50,000 |
| London | 100,000 |
| Exeter | 40,000 |
| Bristol | 30,000 |
| Northampton | 25,000 |

We can actually only choose four of the six depots to open.  Opening a depot has a cost:

| Depot | Cost to open |
| --- | --- |
| Newcastle | 10,000 |
| Exeter | 5,000 |
| Bristol | 12,000 |
| Northampton | 4,000 |

(Note that the description in the book talks about the cost of opening Bristol or Northampton, and the savings from closing Newcastle or Exeter, but these are simply different ways of phrasing the same choice).

We also have the option of expanding the capacity at Birmingham by 20,000 tons, for a cost of \$3000.

Our network has six customers, each with a given demand.

| Customer | Demand (tons) |
| --- | --- |
| C1 | 50,000 |
| C2 | 10,000 |
| C3 | 40,000 |
| C4 | 35,000 |
| C5 | 60,000 |
| C6 | 20,000 |

Shipping costs are given in the following table (in dollars per ton).  Columns are source cities and rows are destination cities.  Thus, for example, it costs $1 per ton to ship the product from Liverpool to London.  A '-' in the table indicates that that combination is not possible, so for example it is not possible to ship from the factory in Brighton to the depot in Newcastle.

| To | Liverpool | Brighton | Newcastle | Birmingham | London | Exeter | Briston | Northhampton
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Depots |
| Newcastle   | 0.5 |   - |
| Birmingham  | 0.5 | 0.3 |
| London      | 1.0 | 0.5 |
| Exeter      | 0.2 | 0.2 |
| Bristol     | 0.6 | 0.4 |
| Northampton | 0.4 | 0.3 |
| Customers |
| C1 | 1.0 | 2.0 |   - | 1.0 |   - |   - | 1.2 |   - |
| C2 |   - |   - | 1.5 | 0.5 | 1.5 |   - | 0.6 | 0.4 |
| C3 | 1.5 |   - | 0.5 | 0.5 | 2.0 | 0.2 | 0.5 |   - |
| C4 | 2.0 |   - | 1.5 | 1.0 |   - | 1.5 |   - | 0.5 |
| C5 |   - |   - |   - | 0.5 | 0.5 | 0.5 | 0.3 | 0.6 |
| C6 | 1.0 |   - | 1.0 |   - | 1.5 | 1.5 | 0.8 | 0.9 |

The questions to be answered: (i) Which four depots should be opened? (ii) Should Birmingham be expanded? (iii) Which depots should be used to satisfy customer demand?

---
## Model Formulation

### Sets and Indices

$f \in \text{Factories}=\{\text{Liverpool}, \text{Brighton}\}$

$d \in \text{Depots}=\{\text{Newcastle}, \text{Birmingham}, \text{London}, \text{Exeter}, \text{Bristol}, \text{Northampton}\}$

$c \in \text{Customers}=\{\text{C1}, \text{C2}, \text{C3}, \text{C4}, \text{C5}, \text{C6}\}$

$\text{Cities} = \text{Factories} \cup \text{Depots} \cup \text{Customers}$

### Parameters

$\text{cost}_{s,t} \in \mathbb{R}^+$: Cost of shipping one ton from source $s$ to destination $t$.

$\text{supply}_f \in \mathbb{R}^+$: Maximum possible supply from factory $f$ (in tons).

$\text{through}_d \in \mathbb{R}^+$: Maximum possible flow through depot $d$ (in tons).

$\text{demand}_c \in \mathbb{R}^+$: Demand for goods at customer $c$ (in tons).

$\text{opencost}_d \in \mathbb{R}^+$: Cost of opening depot $d$ (in dollars).

### Decision Variables

$\text{flow}_{s,t} \in \mathbb{N}^+$: Quantity of goods (in tons) that is shipped from source $s$ to destionation $t$.

$\text{open}_{d} \in [0,1]$: Is depot $d$ open?

$\text{expand} \in [0,1]$: Should Birmingham be expanded?


### Objective Function

- **Cost**: Minimize total shipping costs plus costs of opening depots.

\begin{equation}
\text{Minimize} \quad Z = \sum_{(s,t) \in \text{Cities} \times \text{Cities}}{\text{cost}_{s,t}*\text{flow}_{s,t}} +
                          \sum_{{d} \in \text{Depots}}{\text{opencost}_d*\text{open}_d} +
                          3000 * \text{expand}
\end{equation}

### Constraints

- **Factory output**: Flow of goods from a factory must respect maximum capacity.

\begin{equation}
\sum_{t \in \text{Cities}}{\text{flow}_{f,t}} \leq \text{supply}_{f} \quad \forall f \in \text{Factories}
\end{equation}

- **Customer demand**: Flow of goods must meet customer demand.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,c}} = \text{demand}_{c} \quad \forall c \in \text{Customers}
\end{equation}

- **Depot flow**: Flow into a depot equals flow out of the depot.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} =
\sum_{t \in \text{Cities}}{\text{flow}_{d,t}}
\quad \forall d \in \text{Depots}
\end{equation}

- **Depot capacity (all but Birmingham)**: Flow into a depot must respect depot capacity, and is only allowed if the depot is open.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} \leq \text{through}_{d} * \text{open}_{d}
\quad \forall d \in \text{Depots} - \text{Birmingham}
\end{equation}

- **Depot capacity (Birmingham)**: Flow into Birmingham must respect depot capacity, which may have been expanded.

\begin{equation}
\sum_{s \in \text{Cities}} \text{flow}_{s,\text{Birmingham}} \leq \text{through}_{\text{Birmingham}} + 20000 * \text{expand}
\end{equation}

- **Open depots**: At most 4 open depots (no choice for Birmingham or London).

\begin{equation}
\sum_{d \in \text{Depots}}{\text{open}_{d}} \leq 4
\end{equation}

\begin{equation}
\text{open}_{\text{Birmingham}} = \text{open}_{\text{London}} = 1
\end{equation}

---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [20]:
%pip install gurobipy

In [21]:
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

## Input Data
We define all the input data for the model.

In [102]:
# Create dictionaries to capture factory supply limits, depot throughput limits, cost of opening depots, and customer demand.

supply = dict({'Liverpool': 150000,
               'Brighton': 200000})

through = dict({'Newcastle': 70000,
                'Birmingham': 50000,
                'London': 100000,
                'Exeter': 40000,
                'Bristol': 30000,
                'Northampton': 25000})

opencost = dict({'Newcastle': 10000,
                 'Birmingham': 0,
                 'London': 0,
                 'Exeter': 5000,
                 'Bristol': 12000,
                 'Northampton': 4000})

demand = dict({'C1': 50000,
               'C2': 10000,
               'C3': 40000,
               'C4': 35000,
               'C5': 60000,
               'C6': 20000})

# Create a dictionary to capture shipping costs.

arcs, cost = gp.multidict({
    ('Liverpool', 'Newcastle'): 0.5,
    ('Liverpool', 'Birmingham'): 0.5,
    ('Liverpool', 'London'): 1.0,
    ('Liverpool', 'Exeter'): 0.2,
    ('Liverpool', 'Bristol'): 0.6,
    ('Liverpool', 'Northampton'): 0.4,
    ('Liverpool', 'C1'): 1.0,
    ('Liverpool', 'C3'): 1.5,
    ('Liverpool', 'C4'): 2.0,
    ('Liverpool', 'C6'): 1.0,
    ('Brighton', 'Birmingham'): 0.3,
    ('Brighton', 'London'): 0.5,
    ('Brighton', 'Exeter'): 0.2,
    ('Brighton', 'Bristol'): 0.4,
    ('Brighton', 'Northampton'): 0.3,
    ('Brighton', 'C1'): 2.0,
    ('Newcastle', 'C2'): 1.5,
    ('Newcastle', 'C3'): 0.5,
    ('Newcastle', 'C5'): 1.5,
    ('Newcastle', 'C6'): 1.0,
    ('Birmingham', 'C1'): 1.0,
    ('Birmingham', 'C2'): 0.5,
    ('Birmingham', 'C3'): 0.5,
    ('Birmingham', 'C4'): 1.0,
    ('Birmingham', 'C5'): 0.5,
    ('London', 'C2'): 1.5,
    ('London', 'C3'): 2.0,
    ('London', 'C5'): 0.5,
    ('London', 'C6'): 1.5,
    ('Exeter', 'C3'): 0.2,
    ('Exeter', 'C4'): 1.5,
    ('Exeter', 'C5'): 0.5,
    ('Exeter', 'C6'): 1.5,
    ('Bristol', 'C1'): 1.2,
    ('Bristol', 'C2'): 0.6,
    ('Bristol', 'C3'): 0.5,
    ('Bristol', 'C5'): 0.3,
    ('Bristol', 'C6'): 0.8,
    ('Northampton', 'C2'): 0.4,
    ('Northampton', 'C4'): 0.5,
    ('Northampton', 'C5'): 0.6,
    ('Northampton', 'C6'): 0.9
})

## Model Deployment

We create a model and the variables. The 'flow' variables simply capture the amount of product that flows along each allowed path between a source and destination.  The 'open' variable capture decisions about which depots to open.  The 'expand' variable captures the choice of whether to expand Birmingham.  Objective coefficients are provided here, so we don't need to provide an optimization objective later.

In [104]:
model = gp.Model('SupplyNetworkDesign2')

depots = through.keys()
flow = model.addVars(arcs, obj=cost, name="flow")
open = model.addVars(depots, obj=opencost, vtype=GRB.BINARY, name="open")
expand = model.addVar(obj=3000, vtype=GRB.BINARY, name="expand")

open['Birmingham'].lb = 1
open['London'].lb = 1
model.objcon = -(opencost['Newcastle'] + opencost['Exeter']) # Phrased as 'savings from closing'

Our first constraints require the total flow along arcs leaving a factory to be at most as large as the supply capacity of that factory.

In [105]:
# Production capacity limits

factories = supply.keys()
factory_flow = model.addConstrs((gp.quicksum(flow.select(factory, '*')) <= supply[factory]
                                 for factory in factories), name="factory")

Our next constraints require the total flow along arcs entering a customer to be equal to the demand from that customer.

In [106]:
# Customer demand

customers = demand.keys()
customer_flow = model.addConstrs((gp.quicksum(flow.select('*', customer)) == demand[customer]
                                  for customer in customers), name="customer")

Our final constraints relate to depots.  The first constraints require that the total amount of product entering the depot must equal the total amount leaving.

In [107]:
# Depot flow conservation

depot_flow = model.addConstrs((gp.quicksum(flow.select(depot, '*')) == gp.quicksum(flow.select('*', depot))
                               for depot in depots), name="depot")

The second set limits the product passing through the depot to be at most equal the throughput of that deport, or 0 if the depot isn't open.

In [108]:
# Depot throughput

all_but_birmingham = list(set(depots) - set(['Birmingham']))

depot_capacity = model.addConstrs((gp.quicksum(flow.select(depot, '*')) <= through[depot]*open[depot]
                                   for depot in all_but_birmingham), name="depot_capacity")


The capacity constraint for Birmingham is different.  The depot is always open, but we have the option of expanding its capacity.

In [109]:
birmingham_capacity = model.addConstr(gp.quicksum(flow.select('*', 'Birmingham')) <= through['Birmingham'] +
                                      20000*expand, name="birmingham_capacity")

Finally, there's a limit of at most 4 open depots

In [110]:
# Depot count

depot_count = model.addConstr(open.sum() <= 4)

We now optimize the model

In [111]:
model.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 21 rows, 49 columns and 119 nonzeros
Model fingerprint: 0xc5cfed62
Variable types: 42 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [2e-01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 2e+05]
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolved: 20 rows, 47 columns, 113 nonzeros
Variable types: 42 continuous, 5 integer (5 binary)
Found heuristic solution: objective 174000.00000

Root relaxation: cutoff, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      1

---
## Analysis

The product demand from all of our customers can be satisfied for a total cost of $\$174,000$ by opening a depot in Northampton, closing the depot in Newcastle, and expanding the depot in Birmingham:

In [112]:
print('List of open depots:', [d for d in depots if open[d].x > 0.5])
if expand.x > 0.5:
    print('Expand Birmingham')

List of open depots: ['Birmingham', 'London', 'Exeter', 'Northampton']
Expand Birmingham


In [113]:
product_flow = pd.DataFrame(
    [{"From": arc[0], "To": arc[1], "Flow": flow[arc].x} for arc in arcs if flow[arc].x > 1e-6]
)
product_flow.index=[''] * len(product_flow)
product_flow

,From,To,Flow
,Liverpool,C1,50000.0
,Liverpool,C6,20000.0
,Brighton,Birmingham,70000.0
,Brighton,London,10000.0
,Brighton,Exeter,40000.0
,Brighton,Northampton,25000.0
,Birmingham,C2,10000.0
,Birmingham,C4,10000.0
,Birmingham,C5,50000.0
,London,C5,10000.0



---
## Problem 2, (a)

Add the constraint that if depot in Exeter is open, then depot in Northampton should be closed


In [114]:
# If the depot in Exeter open, then depot in Northampton closed
depot_exeter_northampton = model.addConstr(open["Exeter"] <= 1 - open["Northampton"])

In [115]:
model.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 22 rows, 49 columns and 121 nonzeros
Model fingerprint: 0x9e5e1810
Variable types: 42 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [2e-01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R21 by 1.000000000

Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolved: 21 rows, 47 columns, 115 nonzeros
Variable types: 42 continuous, 5 integer (5 binary)
Found heuristic solution: objective 196000.00000
Found heuristic solution: objective 187500.00000

Root relaxation: cutoff, 21 iterations, 0.00 seconds (0.00 work uni

In [116]:
print('List of open depots:', [d for d in depots if open[d].x > 0.5])
if expand.x > 0.5:
    print('Expand Birmingham')

List of open depots: ['Birmingham', 'London', 'Exeter']
Expand Birmingham


In [117]:
if model.status == GRB.OPTIMAL:
    print("Optimal objective value:", model.ObjVal)
else:
    print("No optimal solution found.")

Optimal objective value: 187500.0


Clean

In [119]:
# Remove the added constraint, so that next example works with the initial model
model.remove(depot_exeter_northampton)

---
## Problem 2, (b)

Add the constraint that either at most 3 depots or exactly all 5 depots (including the depots in Birmingham and London) should be opened.<br>

Create a binary variable, which is 0 if at most 3 depots are opened and 1 if 5 depots are opened.


In [120]:
# First remove the initial depot_count constraint of maximum 4 open depots
model.remove(depot_count)

# Create a binary variable open_3_or_5, which is 0 if at most 3 are opened and 1 otherwise
open_3_or_5 = model.addVar(vtype=GRB.BINARY, name="open_3_or_5")

# Add constraints
open_3_or_5_first_constraint = model.addConstr(open.sum() <= 3 + 2 * open_3_or_5)
open_3_or_5_second_constraint = model.addConstr(open.sum() >= 5 * open_3_or_5)

In [121]:
model.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 22 rows, 50 columns and 127 nonzeros
Model fingerprint: 0xf4638c95
Variable types: 42 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [2e-01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+05]

MIP start from previous solve produced solution with objective 187500 (0.01s)
Loaded MIP start from previous solve with objective 187500

Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolved: 21 rows, 48 columns, 119 nonzeros
Variable types: 42 continuous, 6 integer (6 binary)

Root relaxation: objective 1.740000e+05, 20 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl U

In [122]:
print('List of open depots:', [d for d in depots if open[d].x > 0.5])
if expand.x > 0.5:
    print('Expand Birmingham')

List of open depots: ['Birmingham', 'London', 'Exeter', 'Bristol', 'Northampton']


In [123]:
if model.status == GRB.OPTIMAL:
    print("Optimal objective value:", model.ObjVal)
else:
    print("No optimal solution found.")

Optimal objective value: 178000.0


Clean

In [124]:
# Remove constraints and additional vars, so that next example works with the initial model
model.remove(open_3_or_5_first_constraint)
model.remove(open_3_or_5_second_constraint)

# Remove created binary variable
model.remove(open_3_or_5)

# Add the initial depot constraint
depot_count = model.addConstr(open.sum() <= 4)

---
## Problem 2, (c)

Add the constraint that if both Exeter and Northampton are open, then you need to pay an additional $40000.

Create a binary variable, showing if both Exeter and Northampton are open. <br>
Add the respective term to the objective function and add the required constraints


In [126]:
open_exeter_northampton = model.addVar(vtype=GRB.BINARY, name="open_exeter_northampton")

# Add the new constraints
open_exeter_northampton_1 = model.addConstr(open_exeter_northampton <= open["Exeter"])
open_exeter_northampton_2 = model.addConstr(open_exeter_northampton <= open["Northampton"])
open_exeter_northampton_3 = model.addConstr(open_exeter_northampton >= open["Exeter"] + open["Northampton"] - 1)

# Add the penalty if both depots are open
model.setObjective(model.getObjective() + 40000 * open_exeter_northampton, GRB.MINIMIZE)

In [127]:
model.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 24 rows, 50 columns and 126 nonzeros
Model fingerprint: 0x92384d55
Variable types: 42 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [2e-01, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]

MIP start from previous solve produced solution with objective 214000 (0.01s)
Loaded MIP start from previous solve with objective 214000

Presolve removed 3 rows and 2 columns
Presolve time: 0.00s
Presolved: 21 rows, 48 columns, 116 nonzeros
Variable types: 42 continuous, 6 integer (6 binary)

Root relaxation: objective 1.875000e+05, 22 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl U

In [128]:
print('List of open depots:', [d for d in depots if open[d].x > 0.5])
if expand.x > 0.5:
    print('Expand Birmingham')

List of open depots: ['Birmingham', 'London', 'Exeter']
Expand Birmingham


In [129]:
if model.status == GRB.OPTIMAL:
    print("Optimal objective value:", model.ObjVal)
else:
    print("No optimal solution found.")

Optimal objective value: 187500.0


Clean

In [130]:
# Remove added vars and constraints, so that next example works with initial model
model.remove(open_exeter_northampton_1)
model.remove(open_exeter_northampton_2)
model.remove(open_exeter_northampton_3)

# Remove the binary variable
model.remove(open_exeter_northampton)

# Remove the 40 000·open_exeter_northampton penalty
model.setObjective(model.getObjective() - 40000 * open_exeter_northampton,
                   GRB.MINIMIZE)

---
## Problem 2, (d)

Add the constraint that if the depot in Northampton is open, then the depot in Newcastle should also be open.

In [132]:
open_northampton_newcastle = model.addConstr(open["Northampton"] <= open["Newcastle"])

In [133]:
model.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 22 rows, 49 columns and 121 nonzeros
Model fingerprint: 0xd16529de
Variable types: 42 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [2e-01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 2e+05]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R28 by 1.000000000

Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolved: 21 rows, 47 columns, 115 nonzeros
Variable types: 42 continuous, 5 integer (5 binary)
Found heuristic solution: objective 190500.00000
Found heuristic solution: objective 187500.00000

Root relaxation: objective 1.857500e+05, 23 iterations, 0.00 second

In [134]:
print('List of open depots:', [d for d in depots if open[d].x > 0.5])
if expand.x > 0.5:
    print('Expand Birmingham')

List of open depots: ['Birmingham', 'London', 'Exeter']
Expand Birmingham


In [135]:
if model.status == GRB.OPTIMAL:
    print("Optimal objective value:", model.ObjVal)
else:
    print("No optimal solution found.")

Optimal objective value: 187500.0


Clean

In [136]:
# Remove added vars and constraints, so that next example works with initial model
model.remove(open_northampton_newcastle)

---
## Problem 2, (e)

Add the constraint that if the depots in both Exeter and Northampton are open, then the depot in Newcastle should be opened as well.

Create a binary variable, showing if both Exeter and Northampton are open. <br>

In [138]:
open_exeter_northampton = model.addVar(vtype=GRB.BINARY, name="open_exeter_northampton")

open_exeter_northampton_1 = model.addConstr(open_exeter_northampton <= open["Exeter"])
open_exeter_northampton_2 = model.addConstr(open_exeter_northampton <= open["Northampton"])
open_exeter_northampton_3 = model.addConstr(open_exeter_northampton >= open["Exeter"] + open["Northampton"] - 1)

# Add implication, that if both exeter and northampton are open, then newcastle should be as well
open_exeter_northampton_newcastle = model.addConstr(open_exeter_northampton <= open["Newcastle"])

In [139]:
model.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 25 rows, 50 columns and 128 nonzeros
Model fingerprint: 0x26da3550
Variable types: 42 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [2e-01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 4 rows and 3 columns
Presolve time: 0.00s
Presolved: 21 rows, 47 columns, 116 nonzeros
Variable types: 42 continuous, 5 integer (5 binary)
Found heuristic solution: objective 196000.00000
Found heuristic solution: objective 187500.00000

Root relaxation: objective 1.857500e+05, 23 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objectiv

In [140]:
print('List of open depots:', [d for d in depots if open[d].x > 0.5])
if expand.x > 0.5:
    print('Expand Birmingham')

List of open depots: ['Birmingham', 'London', 'Exeter']
Expand Birmingham


In [141]:
if model.status == GRB.OPTIMAL:
    print("Optimal objective value:", model.ObjVal)
else:
    print("No optimal solution found.")

Optimal objective value: 187500.0


Clean

In [142]:
# Remove created constraints and variables, to work with initial model in the next example
model.remove(open_exeter_northampton_1)
model.remove(open_exeter_northampton_2)
model.remove(open_exeter_northampton_3)

model.remove(open_exeter_northampton_newcastle)

model.remove(open_exeter_northampton)

---
## Problem 2, (f)

Add the constraint that if the depot in Bristol is open, then you can expend its capacity by 10,000 tons, for an additional cost of $700.

In [143]:
# Create a binary variable expand_bristol indicating if Bristol is expanded
expand_bristol = model.addVar(vtype=GRB.BINARY, name="expand_bristol")

# Adjust objective value function
model.setObjective(model.getObjective() + 700 * expand_bristol, GRB.MINIMIZE)

# Adjust throughput
all_but_birmingham_bristol = list(set(depots) - set(['Birmingham'])-set(['Bristol']))

depot_capacity = model.addConstrs((gp.quicksum(flow.select(depot, '*')) <= through[depot]*open[depot]
                                   for depot in all_but_birmingham_bristol), name="depot_capacity")

# Adjust bristol capacity
bristol_capacity = model.addConstr(gp.quicksum(flow.select('*', 'Bristol')) <= through['Bristol'] +
                                      10000*expand, name="bristol_capacity")

# Bristol could be expanded only if it is opened
expand_only_if_open_bristol = model.addConstr(open["Bristol"] <= expand_bristol)

In [144]:
model.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 27 rows, 50 columns and 144 nonzeros
Model fingerprint: 0xba714dd3
Variable types: 42 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [2e-01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 2e+05]
Presolve removed 6 rows and 3 columns
Presolve time: 0.00s
Presolved: 21 rows, 47 columns, 117 nonzeros
Variable types: 42 continuous, 5 integer (5 binary)
Found heuristic solution: objective 196700.00000
Found heuristic solution: objective 191200.00000

Root relaxation: objective 1.740000e+05, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent   

In [145]:
print('List of open depots:', [d for d in depots if open[d].x > 0.5])
if expand.x > 0.5:
    print('Expand Birmingham')

List of open depots: ['Birmingham', 'London', 'Exeter', 'Northampton']
Expand Birmingham


In [146]:
if model.status == GRB.OPTIMAL:
    print("Optimal objective value:", model.ObjVal)
else:
    print("No optimal solution found.")

Optimal objective value: 174000.0
